# Agents 

Agents use LLMs to determine which action to take and what order.

[list of agents](https://langchain.readthedocs.io/en/latest/modules/agents/agents.html)

## Tools

Functions that can be used by agents to interact with the world

[list of tools](https://langchain.readthedocs.io/en/latest/modules/agents/tools.html)



# Getting Started

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["searx-search", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description",
                         verbose=True)

In [1]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: SearX Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Leonardo DiCaprio & girlfriend Camila Morrone couple up for a lunch date! Halle Berry showed off her superhuman strength while carrying all of her groceries to the car in one go during a...
Thought: I need to find out Camila Morrone's age
Action: SearX Search
Action Input: "Camila Morrone age"
Observation: Camila Rebeca Morrone Polak (born June 16, 1997) is an Argentine-American model and actress. She made her acting debut in the James Franco film Bukowski and subsequently appeared in the films Death Wish and Never Goin' Back , both premiering at the Sundance Film Festival in January 2018.

Camila Morrone - Age, Family, Bio | Famous Birthdays Age: 16 Age: 22 Age: 22 Age: 16 Camila Morrone Model #6084 Most Popular Boost Birthday June 16, 1997 Birthplace Buenos Aires , Ar

'Camila Morrone is 24 years old and her age raised to the 0.43 power is approximately 8.'

# Custom Tools 

- agents can receive a list of tools
- tool consists of: name (required), description (optional for some agents),
  return_direct (bool=False)

- The function to be called when a tool is selected should take as input a
  single string and return a single string

- There are two ways to define a tool

In [ ]:
# required imports
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SearxSearchWrapper

In [ ]:
# initialize LLM
llm = OpenAI(temperature=0)

## Completely New Tools

create a new tool from scratch by subclassing `BaseTool`

In [ ]:
# load tool configs that are needed
search = SearxSearchWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
# llm_math.run("what is 13 raised to the .3432 power ?")

tools = [
        Tool(
            name = "Search",
            func=search.run,
            description="a meta search engine. useful for when you need to answer general questions"
            ),
        Tool(
            name="Calculator",
            func=llm_math_chain.run,
            description="useful for when you need to answer questions about math"
            )
        ]

In [ ]:
# constuct agent, we use the default agent type
agent = initialize_agent(tools, llm, agent="zero-shot-react-description",
                         verbose=True)

In [2]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio's girlfriend"
Observation: Camila Morrone (2017 – 2022) Leo, 47, and Camila, 25, a model and actor who stars in Amazon Prime Video's Daisy Jones and the Six series, were first linked in December 2017 after Leo was seen leaving her house in Los Angeles.Dec 29, 2022
Thought: I need to calculate her age raised to the 0.43 power
Action: Calculator
Action Input: 25^0.43

> Entering new LLMMathChain chain...
25^0.43
```python
import math
print(math.pow(25, 0.43))
```

Answer: 3.991298452658078

> Finished chain.

Observation: Answer: 3.991298452658078

Thought: I now know the final answer
Final Answer: Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


"Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 3.991298452658078."

In [ ]:
search_repo = SearxSearchWrapper(query_suffix="site:github.com", k=30)

tools = [
        Tool(
            name = "SearchRepo",
            func=lambda x: search_repo.results(x, num_results=5),
            description="a source code and repository search engine. Useful for \
            when you want to find projects, source code and programming libraries"
            ),
        Tool(
            name = "Search",
            func=search.run,
            description="a meta search engine. useful for when you need to lookup for \
            things on the internet and answer general questions"
            ),
        Tool(
            name="Calculator",
            func=llm_math_chain.run,
            description="useful for when you need to answer questions about math"
            )
        ]

In [ ]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description",
                         verbose=True)

In [3]:
agent.run("I am looking for a python library to build and interact with large \
          language models, give me at most five choices.")



> Entering new AgentExecutor chain...
 I need to find a python library that can do this.
Action: SearchRepo
Action Input: "python library large language models"
Observation: [{'snippet': 'Introduction. LAVIS is a Python deep learning library for LAnguage-and-VISion intelligence research and applications. This library aims to provide engineers and researchers with a one-stop solution to rapidly develop models for their specific multimodal scenarios, and benchmark them across standard and customized datasets. It features a unified ...', 'title': 'LAVIS - A Library for Language-Vision Intelligence - GitHub', 'link': 'https://github.com/salesforce/LAVIS', 'engines': ['duckduckgo', 'qwant', 'bing'], 'category': 'general'}, {'snippet': 'Haystack is an open source NLP framework to interact with your data using Transformer models and LLMs (GPT-3 and alike). Haystack offers production-ready tools to quickly build ChatGPT-like question answering, semantic search, text generation, and more. pyt

'The five python libraries that can build and interact with large language models are LAVIS, Haystack, Whisper, loralib, and LoRA.'

In [4]:
prompt = "Print the github link to some <awesome lists> on the topic of TUIs"

agent.run(prompt)



> Entering new AgentExecutor chain...
 I need to find a github repository with awesome lists on TUIs
Action: SearchRepo
Action Input: "awesome lists" TUIs
Observation: [{'snippet': 'micro A modern and intuitive terminal-based text editor. o Text editor limited by VT100, suitable for programming, writing git commit messages and editing Markdown. slap Sublime-like terminal-based text editor. turbo An experimental text editor for the terminal, based on Scintilla and Turbo Vision.', 'title': 'GitHub - rothgar/awesome-tuis: List of projects that provide …', 'link': 'https://github.com/rothgar/awesome-tuis', 'engines': ['duckduckgo', 'qwant', 'bing', 'google'], 'category': 'general'}, {'snippet': '2 days ago · A curated list of awesome Go frameworks, libraries and software go golang awesome awesome-list golang-library hacktoberfest Updated 10 hours ago Go trimstray / the-book-of-secret-knowledge Star 89.7k Code Issues Pull requests A collection of inspiring lists, manuals, cheatsheets, blo

'https://github.com/rothgar/awesome-tuis'